# Notebook Objective and Setup

BGG01 involves the acquisition of game data from BoardGameGeek. Largely this is accomplished by XML API call, with some dynamic content scraped. Files are dumped to a "dirty" directory.

## Package Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from bs4 import BeautifulSoup
import requests
import regex as re
import time
import json
import os
import gc

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os


## Functions

In [2]:
def create_designers(game_page, game_id):
    '''Create DataFrame for Designers for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all designers on page
    all_designers = game_page.find_all('link', type='boardgamedesigner')
    
    # make dataframe
    designers = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    design = {'BGGId':int(game_id)}
    
    # add this item's designers to dictionary
    for item in all_designers:
        design[item['value']] = int(1)
    
    # append to dataframe
    designers = designers.append(design, ignore_index=True)
    
    # return dataframe
    return designers

In [3]:
def create_categories(game_page, game_id):
    '''Create DataFrame for Categories for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all categories on page
    all_categories = game_page.find_all('link', type='boardgamecategory')
    
    # make dataframe
    categories = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    category = {'BGGId':int(game_id)}
    
    # add this item's categories to dictionary
    for item in all_categories:
        category[item['value']] = int(1)
    
    # append to dataframe
    categories = categories.append(category, ignore_index=True)
    
    # return dataframe
    return categories

In [4]:
def create_mechanics(game_page, game_id):
    '''Create DataFrame for Mechanics for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all mechanics on page
    all_mechanics = game_page.find_all('link', type='boardgamemechanic')
    # make dataframe
    mechanics = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    mechanic = {'BGGId':int(game_id)}

    # add this item's mechanics to dictionary
    for item in all_mechanics:
        mechanic[item['value']] = int(1)
    
    # Try Tableau
    try:
        game_page.find('link', type='boardgamefamily', value=("Mechanism: Tableau Building"))['value']
        mechanic['TableauBuilding'] = int(1)
    except: pass
    
    # Try is Legacy
    try:
        game_page.find('link', type='boardgamefamily', value=("Mechanism: Legacy"))['value']
        mechanic['Legacy'] = int(1)
    except: pass
    
    # append to dataframe
    mechanics = mechanics.append(mechanic, ignore_index=True)
    # return dataframe
    return mechanics

In [5]:
def create_artists(game_page, game_id):
    '''Create DataFrame for artists for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all artists on page
    all_artists = game_page.find_all('link', type='boardgameartist')
    
    # make dataframe
    artists = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    artist = {'BGGId':int(game_id)}

    # add this item's artists to dictionary
    for item in all_artists:
        artist[item['value']] = int(1)
    
    # append to dataframe
    artists = artists.append(artist, ignore_index=True)
    
    # return dataframe
    return artists

In [6]:
def create_publishers(game_page, game_id):
    '''Create DataFrame for Mechanics for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all publishers on page
    all_publishers = game_page.find_all('link', type='boardgamepublisher')
    
    # make dataframe
    publishers = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    publisher = {'BGGId':int(game_id)}
    
    # add this item's artists to dictionary
    for item in all_publishers:
        publisher[item['value']] = int(1)
    
    # append to dataframe
    publishers = publishers.append(publisher, ignore_index=True)
    
    # return dataframe
    return publishers

In [7]:
def create_awards(awards_level, game_id):
    '''Create DataFrame for Awards for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe'''
    
    # find all awards on page
    all_awards = awards_level.find_all('a', class_='ng-binding')
    
    # make dataframe
    awards = pd.DataFrame(columns=['BGGId'])
    # make dictionary for this item
    award = {'BGGId':int(game_id)}

    # add this item's awards to dictionary
    for item in all_awards:
        item = re.sub("[0-9]", "", item.text).strip(' ')
        award[item] = int(1)
    
    # append to dataframe
    awards = awards.append(award, ignore_index=True)
    
    # return dataframe
    return awards

In [8]:
def create_ratings_dist(stats_page, game_id):
    '''Create DataFrame for ratings ditribution for a specific game id
    
    Inputs:
    game_page: page loaded and read with BeautifulSoup
    game_id: id for this game
    
    Outputs:
    dataframe
    '''
    
    # find all ratings sections
    all_ratings = stats_page.find('ratings-stats-graph')
    # find all ratings entries
    next_ratings = all_ratings.find_all('text')
    
    # make dataframe
    ratings = pd.DataFrame(columns=['BGGId'])
    # make dictionary
    rating = {'BGGId':int(game_id)}
    
    # set all ratings in dictionary
    rating['1'] = next_ratings[10].text
    rating['2'] = next_ratings[11].text
    rating['3'] = next_ratings[12].text
    rating['4'] = next_ratings[13].text
    rating['5'] = next_ratings[14].text
    rating['6'] = next_ratings[15].text
    rating['7'] = next_ratings[16].text
    rating['8'] = next_ratings[17].text
    rating['9'] = next_ratings[18].text
    rating['10'] = next_ratings[19].text
    
    # append to dataframe
    ratings = ratings.append(rating, ignore_index=True)
    
    # return dataframe
    return ratings

# Game Scraping

In [9]:
# set up our columns list
columns = ['BGGId',
                'Name',
               'Description',
                'YearPublished',
                'GameWeight',
                'AvgRating',  
                'BayesAvgRating',
                'StdDev',
                'MinPlayers',
                'MaxPlayers',
                'ComAgeRec',
                'LanguageEase',
                'BestPlayers',
                'GoodPlayers',
                'NumOwned',
                'NumWant',
                'NumWish',
                'NumWeightVotes',
                'MfgPlaytime',
                'ComMinPlaytime',
                'ComMaxPlaytime',
                'MfgAgeRec',
                'NumUserRatings',
                'NumComments',
                'NumAlternates',
                'NumExpansions',
                'NumImplementations',
           'IsReimplementation',
                'Family',
                'Theme',
               'Category',
               'Kickstarted',
               'ImagePath',
          ]

Last game id: 349161

In [ ]:
game_ids_current = pd.read_pickle('data_dirty/game_ids_current')
game_ids = list(game_ids_current)

In [ ]:
start_position =0
end_position = 1000
file_suffix = 0

overall_start = time.time()
while end_position < (len(game_ids)+1):
    
    games = pd.DataFrame(columns=columns)
    designers = pd.DataFrame(columns=['BGGId'])
    categories = pd.DataFrame(columns=['BGGId'])
    mechanics = pd.DataFrame(columns=['BGGId'])
    artists = pd.DataFrame(columns=['BGGId'])
    publishers = pd.DataFrame(columns=['BGGId'])
    subcategories = pd.DataFrame(columns=['BGGId'])
    comments = pd.DataFrame(columns=['BGGId'])
    
    ##### File Setup Section #####
    
    # increment file suffix
    file_suffix += 1
    # get file suffix as string
    suffix_str = str(file_suffix)
    
    # print start and end positions
    print("Getting items "+str(start_position+1)+' through '+str(end_position))
    
    # get list of game ids to grab
    #grab_list = game_ids[0][start_position:end_position]
    grab_list = game_ids[start_position:end_position]
    
    # piece together target string of game ids for BGG 
    targets = ''
    for item in grab_list:
        targets += str(item)+','
    
    # log start time for information retrieval
    start = time.time()# log the start time for this entry   
    
    
    ##### API Call Section #####
    
    # Set up Selenium drivers
    options = webdriver.ChromeOptions() # set up chrome options
    options.add_argument("--headless") # set up chrome options
    time.sleep(1) # wait 1 second
    # establish path with targets
    path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&stats=1&type=boardgame'#&comments=1'&ratingcomments=1&page=1&pagesize=100
    driver = webdriver.Chrome(options=options)# initiate chrome driver with options
    print("New page retrieval. May be waiting for load.")
    driver.get(path)# get path
    # wait until the driver finds the element that we need
    element = WebDriverWait(driver, 180).until(EC.presence_of_all_elements_located((By.ID, 'folder0')))
        
    game_page = BeautifulSoup(driver.page_source) # parse page with beautifulsoup    
    
    # make entry for each game item on page
    game_entries = game_page.find_all('item')
    
    print("Items loaded. Processing.")
    ##### Process Each Game #####
    
    for entry in game_entries:
        ##### Get Game Name, BGGId, and check that game should be included in list #####

        ##### Check is expansion #####
        #gametype = entry['type'] # check game type
        #if gametype != 'boardgame':
        #    continue
        #else: pass
       

        # check that this game has sufficient user ratings to incluide
        try:
            user_ratings = int(entry.find('usersrated')['value'])# get the number of user ratings
        
            if user_ratings < 30: #check if user ratings are under 30
                continue
        except: continue
            
        # get game name and BGG ID
        game_name = entry.find('name', type='primary')['value']
        game_id = entry['id']
        #print("Name: "+game_name+", BGG ID: "+str(game_id))

        
        ##### Get Basic Stats #####

        #print("Getting basic stats")
        description = entry.find('description').text # description text of the game
        
        try:
            year_pub = int(entry.find('yearpublished')['value']) # year published
            if year_pub > 2021:
                continue
        except: pass
            
        try: minplayers = int(entry.find('minplayers')['value']) # minimum players
        except: minplayers = None
            
        try: maxplayers = int(entry.find('maxplayers')['value']) # maximum players
        except: maxplayers = None
            
        avg_rating = float(entry.find('average')['value']) # average rating
        bayes_avg = float(entry.find('bayesaverage')['value']) # bayes average rating
        std_dev = float(entry.find('stddev')['value']) # standard deviation of rating
        num_owned = int(entry.find('owned')['value']) # num of people own this game
        num_want = int(entry.find('wanting')['value']) # num of people want this game
        num_wish = int(entry.find('wishing')['value']) # num of people with game on wishlist
        num_weight_votes = int(entry.find('numweights')['value']) # num of votes for game weight
        game_weight = float(entry.find('averageweight')['value']) # voted game weight
        
        try: image_path = entry.find('image').text # path to image
        except: image_path = None
            
        try:  mfg_play_time = int(entry.find('playingtime')['value']) # mfg stated playtime
        except: mfg_play_time = None
        try: comm_min_play = int(entry.find('minplaytime')['value']) # community min playtime
        except: comm_min_play = None
            
        try: comm_max_play = int(entry.find('maxplaytime')['value']) # community max playtime
        except: comm_max_play = None
        
        try: mfg_age = int(entry.find('minage')['value']) # mfg min age
        except: mfg_age = None
            
        #num_comments = int(entry.find('comments')['totalitems']) # num of ratings comments
        num_alts = len(entry.find_all('name', type='alternate')) # number alternate versions
        num_expansions = len(entry.find_all('link', type='boardgameexpansion')) # number of expansions
        num_implementations = len(entry.find_all('link', type='boardgameimplementation')) # number of implementations    
        

        
        ##### Get reimplementation flag #####
        reimplementation = entry.find('link', type="boardgameimplementation", inbound="true") # check if game is a reimplementation
        if reimplementation: 
            reimplements = 1 # if it's a reimplementation, flag it 1
        else: 
            reimplements = 0
 


        ##### Basic stats requiring some compaction/refinement #####

        # community age min
        try:
            age_poll = entry.find('poll', title="User Suggested Player Age").find_all('result')

            total = 0
            items = 0
        
            for item in age_poll:   
                vote = int(item['numvotes']) * int(item['value'][:2])
                total += vote
                items += int(item['numvotes'])

            if items>0: comm_age = total/items # make sure not dividing by 0, get community recommended age
            else: comm_age=None # if no votes, record none
        except: comm_age=None
        
        # Language Ease
        try:
         
            lang_poll = entry.find('poll', title="Language Dependence").find_all('result')
            total, items = 0, 0

            for item in lang_poll:   
                vote = int(item['numvotes']) * int(item['level'])
                total += vote
                items += int(item['numvotes'])

            if items>0: lang_ease = total/items # make sure not dividing by 0, get community language ease
            else: lang_ease=None # if no votes, record none
        except: lang_ease=None # if no votes, record none
            
        try:    
            # Best and Good Players
            players = entry.find('poll', title="User Suggested Number of Players").find_all('results') # get user players poll
            player_num_votes = int(entry.find('poll', title="User Suggested Number of Players")['totalvotes'])# get total votes
        
            best_players, best_score, good_players = 0, 0, [] # set up for best players loop
        
            if player_num_votes > 30: # evaluate if more than 30 votes for num players
                for player in players:
                    best = int(player.find('result', value='Best')['numvotes'])
                    rec = int(player.find('result', value='Recommended')['numvotes'])
                    score = best*2 + rec*1
                    positives = best+rec
                    ratio = positives/player_num_votes
                    if score > best_score: best_players, best_score = player['numplayers'], score # put in # players for best score
                    if ratio > .5: good_players.append(player['numplayers']) # put in good players if over 50% ratio
            else: best_players=None
        except: best_players=None
        
        ##### Skip dynamic content which cannot be batched #####
            
        #this_game['NumFans']=int(num_fans),
        #this_game['NumPageViews']=int(num_views),
        #this_game['RulesPosts']=int(rules_threads),
        #this_game['TotalPosts']=int(total_threads),            
        #this_game['NumAwards'] = int(num_awards)              
            
        
        # make dataframe for this game
        this_game = pd.DataFrame()
        this_game['BGGId']=int(game_id),
        this_game['Name']=game_name,
        this_game['Description']=description,
        this_game['YearPublished']=int(year_pub),
        this_game['GameWeight']=float(game_weight),
        this_game['AvgRating']=float(avg_rating),
        this_game['BayesAvgRating']=float(bayes_avg),
        this_game['StdDev']=float(std_dev),
        this_game['MinPlayers']=minplayers,
        this_game['MaxPlayers']=maxplayers,
        try: this_game['ComAgeRec']=float(comm_age),
        except: this_game['ComAgeRec']=None,
        try: this_game['LanguageEase']=float(lang_ease),
        except: this_game['LanguageEase']=None,
        this_game['BestPlayers']=best_players,
        this_game['GoodPlayers']=good_players,
        this_game['NumOwned']=int(num_owned),
        this_game['NumWant']=int(num_want),
        this_game['NumWish']=int(num_wish),
        this_game['NumWeightVotes']=int(num_weight_votes),
        this_game['MfgPlaytime']=mfg_play_time,
        this_game['ComMinPlaytime']=comm_min_play,
        this_game['ComMaxPlaytime']=comm_max_play,
        this_game['MfgAgeRec']=mfg_age,
        this_game['NumUserRatings']=int(user_ratings),
        #this_game['NumComments']=int(num_comments),
        this_game['NumAlternates']=int(num_alts),
        this_game['NumExpansions']=int(num_expansions),
        this_game['NumImplementations']=int(num_implementations),
        this_game['IsReimplementation']=int(reimplements),
        this_game['ImagePath']=image_path
            
        
        # add unique information to end of df
        
        # Add game ranks
        ranks = entry.find_all('rank')
        try:
            for item in ranks:
                this_game['Rank:'+item['name']] = float(item['value'])
        except: pass
        
        # Try to add components
        try: 
            families = entry.find_all('link', type='boardgamefamily', value=re.compile("Component"))
            for item in families:                    
                this_game['Components:'+item['name']] = item['value']
        except: pass
            
        
        # Try to add game series/family
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Game:"))['value'].strip('Game:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            family = entry.find('link', type='boardgamefamily', value=re.compile("Series:"))['value'].strip('Series:').strip(' ')
            this_game['Family'] = family
        except: pass
            
        try:
            setting = entry.find('link', type='boardgamefamily', value=re.compile("Setting:"))['value'].strip('Setting:').strip(' ')
            this_game['Setting'] = setting
        except: pass
            
        
        # Try to add theme
        try:
            theme = entry.find('link', type='boardgamefamily', value=re.compile("Theme:"))['value'].strip('Theme:').strip(' ')
            this_game['Theme'] = theme
        except: pass
            
        try:
            mechanism = entry.find('link', type='boardgamefamily', value=re.compile("Mechanism:"))['value'].strip('Mechanism:').strip(' ')
            this_game['Mechanism'] = mechanism
        except: pass
        
        # Try to add game category
        try:
            category = entry.find('link', type='boardgamefamily', value=re.compile("Category:"))['value'].strip('Category:').strip(' ')
            this_game['Category'] = category
        except: pass
        
        
        # Try is Kickstarted
        try:
            entry.find('link', type='boardgamefamily', value=re.compile("Crowdfunding"))['value']
            this_game['Kickstarted'] = int(1)
        except: pass
     
    
    
        ##### Get subcategories #####
        
        all_subcategories = entry.find_all('link', type='boardgamecategory')
        
        categories_hold = pd.DataFrame(columns=['BGGId'])
        subcategory = {'BGGId':int(game_id)}
    
        for item in all_subcategories:
            subcategory[item['value']] = int(1)      
        
        categories_hold = categories_hold.append(subcategory, ignore_index=True)    
        
        
        # create specialty dataframes
        designer = create_designers(entry, game_id)
        category = create_categories(entry, game_id)
        mechanic = create_mechanics(entry, game_id)
        artist = create_artists(entry, game_id)
        publisher = create_publishers(entry, game_id)
            
        games = games.append(this_game, ignore_index = True)
        designers = designers.append(designer, ignore_index=True)
        categories = categories.append(category, ignore_index=True)
        mechanics = mechanics.append(mechanic, ignore_index=True)
        artists = artists.append(artist, ignore_index=True)
        publishers = publishers.append(publisher, ignore_index=True)
        subcategories = subcategories.append(categories_hold, ignore_index=True)
        
    games.to_pickle('data_dirty/games'+suffix_str+'.pkl')
    designers.to_pickle('data_dirty/designers'+suffix_str+'.pkl')
    categories.to_pickle('data_dirty/categories'+suffix_str+'.pkl')
    mechanics.to_pickle('data_dirty/mechanics'+suffix_str+'.pkl')
    artists.to_pickle('data_dirty/artists'+suffix_str+'.pkl')
    publishers.to_pickle('data_dirty/publishers'+suffix_str+'.pkl')
    subcategories.to_pickle('data_dirty/subcategories'+suffix_str+'.pkl')
    
    print("Finished items in this group")
    
    print(f'Time: {time.time() - start}\n\n')       
    
    start_position += 1000
    end_position += 1000
    
print(f'Time: {time.time() - overall_start}\n\n') 

### DataValidation

In [ ]:
subcategories1 = pd.read_pickle('data_dirty/subcategories52.pkl')
games1 = pd.read_pickle('data_dirty/games52.pkl')
designers1 = pd.read_pickle('data_dirty/designers52.pkl')
categories1 = pd.read_pickle('data_dirty/categories52.pkl')
mechanics1 = pd.read_pickle('data_dirty/mechanics52.pkl')
artists1 = pd.read_pickle('data_dirty/artists52.pkl')
publishers1 = pd.read_pickle('data_dirty/publishers52.pkl')

In [ ]:
subcategories1

In [ ]:
games1

In [ ]:
designers1

In [ ]:
categories1

In [ ]:
mechanics1

In [ ]:
artists1

In [ ]:
publishers1

## Combine Files

In [ ]:
#games = pd.DataFrame(columns=columns)
#designers = pd.DataFrame(columns=['BGGId'])
#categories = pd.DataFrame(columns=['BGGId'])
#mechanics = pd.DataFrame(columns=['BGGId'])
#artists = pd.DataFrame(columns=['BGGId'])
#publishers = pd.DataFrame(columns=['BGGId'])
#subcategories = pd.DataFrame(columns=['BGGId'])

In [ ]:
for number in range(1, 354):
    print(number)
    
    #this_games = pd.read_pickle('data_dirty/games'+str(number)+'.pkl')
    #this_designers = pd.read_pickle('data_dirty/designers'+str(number)+'.pkl')
    #this_categories = pd.read_pickle('data_dirty/categories'+str(number)+'.pkl')
    #this_mechanics = pd.read_pickle('data_dirty/mechanics'+str(number)+'.pkl')
    #this_artists = pd.read_pickle('data_dirty/artists'+str(number)+'.pkl')
    #this_publishers = pd.read_pickle('data_dirty/publishers'+str(number)+'.pkl')
    #this_subcategories = pd.read_pickle('data_dirty/subcategories'+str(number)+'.pkl')
    
    #games = games.append(this_games)
    #designers = designers.append(this_designers)
    #categories = categories.append(this_categories)
    #mechanics = mechanics.append(this_mechanics)
    #artists = artists.append(this_artists)
    #publishers = publishers.append(this_publishers)
    #subcategories = subcategories.append(this_subcategories)
    
    

In [ ]:
#games = games.reset_index(drop=True)
#designers = designers.reset_index(drop=True)
#categories = categories.reset_index(drop=True)
#mechanics = mechanics.reset_index(drop=True)
#artists = artists.reset_index(drop=True)
#publishers = publishers.reset_index(drop=True)
#subcategories = subcategories.reset_index(drop=True)

In [ ]:
#games.to_pickle('data_dirty/games.pkl')
#designers.to_pickle('data_dirty/designers.pkl')
#categories.to_pickle('data_dirty/categories.pkl')
#mechanics.to_pickle('data_dirty/mechanics.pkl')
#artists.to_pickle('data_dirty/artists.pkl')
#publishers.to_pickle('data_dirty/publishers.pkl')
#subcategories.to_pickle('data_dirty/subcategories.pkl')

In [ ]:
break

# Ratings and Comments

In [9]:
games=pd.read_pickle('data_dirty/games.pkl')

In [10]:
game_ids_current = pd.read_pickle('data_dirty/game_ids_current')
game_ids = list(game_ids_current)

In [11]:
ratings_totals = pd.DataFrame(games['BGGId'])
ratings_totals['RatingsPages'] = np.ceil(games['NumUserRatings']/100)
ratings_totals = ratings_totals.sort_values('RatingsPages', ascending=False).reset_index(drop=True)

In [12]:
ratings_totals.head(30)

,BGGId,RatingsPages
0,30549,1082
1,822,1080
2,13,1072
3,68448,893
4,36218,812
5,9209,756
6,178900,736
7,167791,731
8,173346,683
9,31260,659


In [54]:
pages_for_item = ratings_totals.set_index('BGGId').to_dict(orient='dict')['RatingsPages']

In [55]:
pages_for_item

{30549: 1082,
 822: 1080,
 13: 1072,
 68448: 893,
 36218: 812,
 9209: 756,
 178900: 736,
 167791: 731,
 173346: 683,
 31260: 659,
 3076: 652,
 40692: 643,
 148228: 640,
 169786: 638,
 14996: 627,
 230802: 616,
 70323: 607,
 129622: 591,
 2651: 584,
 266192: 545,
 39856: 540,
 163412: 515,
 84876: 501,
 478: 500,
 28143: 487,
 110327: 484,
 174430: 472,
 34635: 470,
 161936: 447,
 65244: 443,
 1927: 435,
 12333: 429,
 120677: 416,
 98778: 413,
 150376: 412,
 54043: 408,
 11: 399,
 10547: 394,
 50: 393,
 131357: 384,
 147020: 383,
 70919: 377,
 133473: 377,
 15987: 375,
 170216: 372,
 41114: 372,
 121921: 368,
 204583: 364,
 181304: 349,
 157354: 343,
 463: 342,
 183394: 342,
 37111: 339,
 126163: 330,
 193738: 326,
 205637: 325,
 201808: 316,
 181: 316,
 2655: 315,
 162886: 315,
 157969: 313,
 146021: 313,
 124361: 312,
 40834: 311,
 194655: 311,
 31481: 308,
 237182: 308,
 171: 306,
 1406: 306,
 103343: 301,
 6249: 301,
 199561: 299,
 102794: 296,
 96848: 293,
 39463: 287,
 244521: 287

In [13]:
highest = ratings_totals['RatingsPages'].max()

## Scrape for multi-page entries

In [56]:
pages_1100 = list(ratings_totals.loc[ratings_totals['RatingsPages']>=1000]['BGGId'])
pages_999 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<1000) & (ratings_totals['RatingsPages']>=600)]['BGGId'])
pages_599 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<600) & (ratings_totals['RatingsPages']>=400)]['BGGId'])
pages_399 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<400) & (ratings_totals['RatingsPages']>=300)]['BGGId'])
pages_299 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<300) & (ratings_totals['RatingsPages']>=200)]['BGGId'])
pages_199 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<200) & (ratings_totals['RatingsPages']>=100)]['BGGId'])
pages_99 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<100) & (ratings_totals['RatingsPages']>=50)]['BGGId'])
pages_49 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<50) & (ratings_totals['RatingsPages']>=40)]['BGGId'])
pages_39 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<40) & (ratings_totals['RatingsPages']>=30)]['BGGId'])
pages_29 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<30) & (ratings_totals['RatingsPages']>=20)]['BGGId'])

pages_19 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<20) & (ratings_totals['RatingsPages']>=12)]['BGGId'])

pages_10 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<12) & (ratings_totals['RatingsPages']>=10)]['BGGId'])

pages_9 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<10) & (ratings_totals['RatingsPages']>=7)]['BGGId'])

In [57]:
list_of_lists = {}
list_of_lists[highest] = pages_1100
list_of_lists[999] = pages_999
list_of_lists[599] = pages_599
list_of_lists[399] = pages_399
list_of_lists[299] = pages_299
list_of_lists[199] = pages_199
list_of_lists[99] = pages_99
list_of_lists[49] = pages_49
list_of_lists[39] = pages_39
list_of_lists[29] = pages_29
list_of_lists[19] = pages_19
list_of_lists[10] = pages_10
list_of_lists[9] = pages_9

In [ ]:
# Set up Selenium drivers
options = webdriver.ChromeOptions() # set up chrome options
options.add_argument("--headless") # set up chrome options       

# for each item in the list:
for item in list_of_lists:
    
    start = time.time()
    
    # make blank dataframe for user ratings for this block of targets
    user_ratings = pd.DataFrame(columns=['BGGId'])
    #user_ratings = pd.read_pickle('data_dirty/user_ratings_comments'+str(item)+'.pkl')
    
    page = int(item)
    
    # get number of ratings pages for this block of targets
    pages = np.arange(1, page+1, 1)
    
    # for each page in the number of ratings pages:
    for page in pages[:]:
              
        # piece together target string of game ids for this block of targets 
        targets = ''
        for target in list_of_lists[item]:
            if pages_for_item[target] >= page:
                targets += str(target)+','
            else: continue

        # report what page we are on
        print("Getting page "+str(page)+" of "+str(pages.max()))
        
        # establish path with targets and current page
        path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&ratingcomments=1&page='+str(page)+'&pagesize=100'
        print(path)
        
        ##### API Call Section #####
        driver = webdriver.Chrome(options=options)# initiate chrome driver with options
        #print(path)
        print("Waiting for page to load")
        
        driver.get(path)# get path
        
        # wait until the driver finds the element that we need
        #myElem = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.ID, 'folder1')))
        
        #element_present = EC.presence_of_element_located((By.ID, 'folder1'))
        #WebDriverWait(driver, 1000).until(element_present)
        
        #print("Page loaded. Parsing with BeautifulSoup.")
        #game_page = BeautifulSoup(driver.page_source) # parse page with beautifulsoup    
    
        # make entry for each game item on page
        
        #game_entries = game_page.find_all('item')
        game_entries = driver.find_elements_by_tag_name('item')
        
        print("Page loaded. Processing.")
        ##### Process Each Game #####
        
        # for each game found on the page:
        for entry in game_entries:
            
            elementHTML = entry.get_attribute('outerHTML') #gives exact HTML content of the element
            elementSoup = BeautifulSoup(elementHTML,'html.parser')
            
            # make an empty dataframe for this item
            this_game_ratings = pd.DataFrame(columns=['BGGId'])
            
            # get the item id and item name
            item_id = int(elementSoup.find('item')['id'])
            item_name = elementSoup.find('name')['value']
        
            # get the list of ratings
            list_of_ratings = elementSoup.find_all('comment')
            
            # set up empty list to store the ratings found on this page
            bggid, names, ratings, comment, usernames = [], [], [], [], []
            
            # for each item in the list of ratings:
            for rating in list_of_ratings:
                # add the raw rating to the raw_ratings dictionary
                # add the item id to the list
                bggid.append(item_id)
                # add the item name to the list
                names.append(item_name)
                # add the rating to the list
                ratings.append(rating['rating'])
                # add the comment to the list
                comment.append(rating['value'])
                # add the username to the list
                usernames.append(rating['username'].strip('_'))
            
            # fill the dataframe for the entry
            this_game_ratings['BGGId'] = bggid    
            this_game_ratings['Name'] = names
            this_game_ratings['Rating'] = ratings
            this_game_ratings['Value'] = comment
            this_game_ratings['Username'] = usernames
            
            # append the entry to the overall user_ratings dataframe
            user_ratings = user_ratings.append(this_game_ratings)
        
        driver.close()
        
        end = time.time()
        print(str(end-start)+" seconds elapsed")
        
    user_ratings.to_pickle('data_dirty/pulled_ratings/user_ratings_comments'+str(item)+'.pkl')
        
    #del game_page
    del driver
    del game_entries
    del elementHTML
    del elementSoup
    del list_of_ratings
    gc.collect() 

Getting page 1 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=1&pagesize=100
Waiting for page to load
Page loaded. Processing.
1.5960485935211182 seconds elapsed
Getting page 2 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=2&pagesize=100
Waiting for page to load
Page loaded. Processing.
3.1014726161956787 seconds elapsed
Getting page 3 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=3&pagesize=100
Waiting for page to load
Page loaded. Processing.
4.659074783325195 seconds elapsed
Getting page 4 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=4&pagesize=100
Waiting for page to load
Page loaded. Processing.
8.292446374893188 seconds elapsed
Getting page 5 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=5&pagesize=100
Waiting for page to load
Page loaded. Processing.
9.82984

Page loaded. Processing.
170.4441635608673 seconds elapsed
Getting page 41 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=41&pagesize=100
Waiting for page to load
Page loaded. Processing.
175.67734146118164 seconds elapsed
Getting page 42 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=42&pagesize=100
Waiting for page to load
Page loaded. Processing.
180.76460766792297 seconds elapsed
Getting page 43 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=43&pagesize=100
Waiting for page to load
Page loaded. Processing.
187.85209012031555 seconds elapsed
Getting page 44 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=44&pagesize=100
Waiting for page to load
Page loaded. Processing.
192.95352721214294 seconds elapsed
Getting page 45 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=45

Waiting for page to load
Page loaded. Processing.
439.85812759399414 seconds elapsed
Getting page 81 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=81&pagesize=100
Waiting for page to load
Page loaded. Processing.
469.11922311782837 seconds elapsed
Getting page 82 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=82&pagesize=100
Waiting for page to load
Page loaded. Processing.
474.45726323127747 seconds elapsed
Getting page 83 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=83&pagesize=100
Waiting for page to load
Page loaded. Processing.
479.66427421569824 seconds elapsed
Getting page 84 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=84&pagesize=100
Waiting for page to load
Page loaded. Processing.
485.1464743614197 seconds elapsed
Getting page 85 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,

Waiting for page to load
Page loaded. Processing.
734.2808454036713 seconds elapsed
Getting page 121 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=121&pagesize=100
Waiting for page to load
Page loaded. Processing.
739.3961064815521 seconds elapsed
Getting page 122 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=122&pagesize=100
Waiting for page to load
Page loaded. Processing.
766.6193561553955 seconds elapsed
Getting page 123 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=123&pagesize=100
Waiting for page to load
Page loaded. Processing.
771.6989345550537 seconds elapsed
Getting page 124 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=124&pagesize=100
Waiting for page to load
Page loaded. Processing.
778.6497149467468 seconds elapsed
Getting page 125 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,82

Waiting for page to load
Page loaded. Processing.
1030.5919942855835 seconds elapsed
Getting page 161 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=161&pagesize=100
Waiting for page to load
Page loaded. Processing.
1038.024881362915 seconds elapsed
Getting page 162 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=162&pagesize=100
Waiting for page to load
Page loaded. Processing.
1043.1912167072296 seconds elapsed
Getting page 163 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=163&pagesize=100
Waiting for page to load
Page loaded. Processing.
1051.0099518299103 seconds elapsed
Getting page 164 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549,822,13,&ratingcomments=1&page=164&pagesize=100
Waiting for page to load
Page loaded. Processing.
1078.291629076004 seconds elapsed
Getting page 165 of 1082
https://www.boardgamegeek.com/xmlapi2/thing?id=30549

...

Waiting for page to load
Page loaded. Processing.
6415.62872171402 seconds elapsed
Getting page 88 of 199
https://www.boardgamegeek.com/xmlapi2/thing?id=116,177736,12,161970,166384,5782,54998,62219,220308,27162,180263,5,21790,12493,218603,2511,198773,54,1294,25613,170042,15512,74,118,92828,43570,37904,156129,124708,205896,113924,171131,167355,244522,25669,209778,127023,160477,28023,163967,62871,239188,194594,24480,118048,216132,147151,258,224037,92415,760,127398,21763,9674,263918,113294,34219,144733,233078,63888,188,102680,169426,312484,115746,175914,3,254640,38453,102652,291457,217372,180974,209010,438,229853,9217,77130,140934,160499,43443,555,823,2471,43111,131835,182874,48726,155703,172287,1917,59959,155821,41,27588,811,156009,225694,12692,63268,46213,140620,20100,286096,215,27627,219513,172818,199042,21241,29368,199478,198994,316554,191189,285774,262712,161417,176189,97207,221107,91536,155362,278,19857,521,185343,25292,45315,45,503,83330,20551,221194,191862,125618,171668,265736,119

Waiting for page to load
Page loaded. Processing.
6962.597636461258 seconds elapsed
Getting page 94 of 199
https://www.boardgamegeek.com/xmlapi2/thing?id=116,177736,12,161970,166384,5782,54998,62219,220308,27162,180263,5,21790,12493,218603,2511,198773,54,1294,25613,170042,15512,74,118,92828,43570,37904,156129,124708,205896,113924,171131,167355,244522,25669,209778,127023,160477,28023,163967,62871,239188,194594,24480,118048,216132,147151,258,224037,92415,760,127398,21763,9674,263918,113294,34219,144733,233078,63888,188,102680,169426,312484,115746,175914,3,254640,38453,102652,291457,217372,180974,209010,438,229853,9217,77130,140934,160499,43443,555,823,2471,43111,131835,182874,48726,155703,172287,1917,59959,155821,41,27588,811,156009,225694,12692,63268,46213,140620,20100,286096,215,27627,219513,172818,199042,21241,29368,199478,198994,316554,191189,285774,262712,161417,176189,97207,221107,91536,155362,278,19857,521,185343,25292,45315,45,503,83330,20551,221194,191862,125618,171668,265736,11

Waiting for page to load
Page loaded. Processing.
7503.473761558533 seconds elapsed
Getting page 100 of 199
https://www.boardgamegeek.com/xmlapi2/thing?id=116,177736,12,161970,166384,5782,54998,62219,220308,27162,180263,5,21790,12493,218603,2511,198773,54,1294,25613,170042,15512,74,118,92828,43570,37904,156129,124708,205896,113924,171131,167355,244522,25669,209778,127023,160477,28023,163967,62871,239188,194594,24480,118048,216132,147151,258,224037,92415,760,127398,21763,9674,263918,113294,34219,144733,233078,63888,188,102680,169426,312484,115746,175914,3,254640,38453,102652,291457,217372,180974,209010,438,229853,9217,77130,140934,160499,43443,555,823,2471,43111,131835,182874,48726,155703,172287,1917,59959,155821,41,27588,811,156009,225694,12692,63268,46213,140620,20100,286096,215,27627,219513,172818,199042,21241,29368,199478,198994,316554,191189,285774,262712,161417,176189,97207,221107,91536,155362,278,19857,521,185343,25292,45315,45,503,83330,20551,221194,191862,125618,171668,265736,1

Waiting for page to load
Page loaded. Processing.
7978.102038860321 seconds elapsed
Getting page 106 of 199
https://www.boardgamegeek.com/xmlapi2/thing?id=116,177736,12,161970,166384,5782,54998,62219,220308,27162,180263,5,21790,12493,218603,2511,198773,54,1294,25613,170042,15512,74,118,92828,43570,37904,156129,124708,205896,113924,171131,167355,244522,25669,209778,127023,160477,28023,163967,62871,239188,194594,24480,118048,216132,147151,258,224037,92415,760,127398,21763,9674,263918,113294,34219,144733,233078,63888,188,102680,169426,312484,115746,175914,3,254640,38453,102652,291457,217372,180974,209010,438,229853,9217,77130,140934,160499,43443,555,823,2471,43111,131835,182874,48726,155703,172287,1917,59959,155821,41,27588,811,156009,225694,12692,63268,46213,140620,20100,286096,215,27627,219513,172818,199042,21241,29368,199478,198994,316554,191189,285774,262712,161417,176189,97207,221107,91536,155362,278,19857,521,185343,25292,45315,45,503,83330,20551,221194,191862,125618,171668,265736,1

Waiting for page to load
Page loaded. Processing.
8493.068077802658 seconds elapsed
Getting page 113 of 199
https://www.boardgamegeek.com/xmlapi2/thing?id=116,177736,12,161970,166384,5782,54998,62219,220308,27162,180263,5,21790,12493,218603,2511,198773,54,1294,25613,170042,15512,74,118,92828,43570,37904,156129,124708,205896,113924,171131,167355,244522,25669,209778,127023,160477,28023,163967,62871,239188,194594,24480,118048,216132,147151,258,224037,92415,760,127398,21763,9674,263918,113294,34219,144733,233078,63888,188,102680,169426,312484,115746,175914,3,254640,38453,102652,291457,217372,180974,209010,438,229853,9217,77130,140934,160499,43443,555,823,2471,43111,131835,182874,48726,155703,172287,1917,59959,155821,41,27588,811,156009,225694,12692,63268,46213,140620,20100,286096,215,27627,219513,172818,199042,21241,29368,199478,198994,316554,191189,285774,262712,161417,176189,97207,221107,91536,155362,278,19857,521,185343,25292,45315,45,503,83330,20551,221194,191862,125618,171668,265736,1

## Scrape for consistent page entries

In [ ]:
pages_6 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<7) & (ratings_totals['RatingsPages']>=6)]['BGGId'])

pages_5 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<6) & (ratings_totals['RatingsPages']>=5)]['BGGId'])

pages_4 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<5) & (ratings_totals['RatingsPages']>=4)]['BGGId'])

pages_3_1 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<4) & (ratings_totals['RatingsPages']>=3)]['BGGId'])[:500]
pages_3_2 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<4) & (ratings_totals['RatingsPages']>=3)]['BGGId'])[500:1000]
pages_3_3 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<4) & (ratings_totals['RatingsPages']>=3)]['BGGId'])[1000:1500]
pages_3_4 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<4) & (ratings_totals['RatingsPages']>=3)]['BGGId'])[1500:]

pages_2_1 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[:500]
pages_2_2 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[500:1000]
pages_2_3 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[1000:1500]
pages_2_4 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[1500:2000]
pages_2_5 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[2000:2500]
pages_2_6 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[2500:3000]
pages_2_7 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[3000:3500]
pages_2_8 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<3) & (ratings_totals['RatingsPages']>=2)]['BGGId'])[3500:]


pages_1_1 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[:500]
pages_1_2 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[500:1000]
pages_1_3 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[1000:1500]
pages_1_4 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[1500:2000]
pages_1_5 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[2000:2500]
pages_1_6 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[2500:3000]
pages_1_7 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[3000:3500]
pages_1_8 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[3500:4000]
pages_1_9 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[4000:4500]
pages_1_10 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[4500:5000]
pages_1_11 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[5000:5500]
pages_1_12 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[5500:6000]
pages_1_13 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[6000:6500]
pages_1_14 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[6500:7000]
pages_1_15 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[7000:7500]
pages_1_16 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[7500:8000]
pages_1_17 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[8000:8500]
pages_1_18 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[8500:9000]
pages_1_19 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[9000:9500]
pages_1_20 = list(ratings_totals.loc[(ratings_totals['RatingsPages']<2)]['BGGId'])[9500:]

In [38]:
list_of_lists2 = {}

list_of_lists2['6'] = pages_6
list_of_lists2['5'] = pages_5
list_of_lists2['4'] = pages_4
list_of_lists2['3_1'] = pages_3_1
list_of_lists2['3_2'] = pages_3_2
list_of_lists2['3_3'] = pages_3_3
list_of_lists2['3_4'] = pages_3_4
list_of_lists2['2_1'] = pages_2_1
list_of_lists2['2_2'] = pages_2_2
list_of_lists2['2_3'] = pages_2_3
list_of_lists2['2_4'] = pages_2_4
list_of_lists2['2_5'] = pages_2_5
list_of_lists2['2_6'] = pages_2_6
list_of_lists2['2_7'] = pages_2_7
list_of_lists2['2_8'] = pages_2_8
list_of_lists2['1_1'] = pages_1_1
list_of_lists2['1_2'] = pages_1_2
list_of_lists2['1_3'] = pages_1_3
list_of_lists2['1_4'] = pages_1_4
list_of_lists2['1_5'] = pages_1_5
list_of_lists2['1_6'] = pages_1_6
list_of_lists2['1_7'] = pages_1_7
list_of_lists2['1_8'] = pages_1_8
list_of_lists2['1_9'] = pages_1_9
list_of_lists2['1_10'] = pages_1_10
list_of_lists2['1_11'] = pages_1_11
list_of_lists2['1_12'] = pages_1_12
list_of_lists2['1_13'] = pages_1_13
list_of_lists2['1_14'] = pages_1_14
list_of_lists2['1_15'] = pages_1_15
list_of_lists2['1_16'] = pages_1_16
list_of_lists2['1_17'] = pages_1_17
list_of_lists2['1_18'] = pages_1_18
list_of_lists2['1_19'] = pages_1_19
list_of_lists2['1_20'] = pages_1_20


#page = int(str(item)[:1])

In [40]:
# Set up Selenium drivers
options = webdriver.ChromeOptions() # set up chrome options
options.add_argument("--headless") # set up chrome options       

for item in list_of_lists2:
    
    print(item)
    
    start = time.time()
    
    # make blank dataframe for user ratings for this block of targets
    user_ratings = pd.DataFrame(columns=['BGGId'])
    #user_ratings = pd.read_pickle('data_dirty/user_ratings_comments'+str(item)+'.pkl')
    
    page = int(item[:1])
    
    # get number of ratings pages for this block of targets
    pages = np.arange(1, page+1, 1)
    
    # piece together target string of game ids for this block of targets 
    targets = ''
    for target in list_of_lists2[item]:
        targets += str(target)+','
    
    #for entry in list_of_lists[item]:
    #    raw_ratings[entry] = []
    
    # for each page in the number of ratings pages:
    for page in pages[:]:
        
        # report what page we are on
        print("Getting page "+str(page)+" of "+str(pages.max()))
        
        # establish path with targets and current page
        path = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+targets+'&ratingcomments=1&page='+str(page)+'&pagesize=100'
        
        ##### API Call Section #####
        driver = webdriver.Chrome(options=options)# initiate chrome driver with options
        #print(path)
        print("Waiting for page to load")
        
        driver.get(path)# get path
        
        # wait until the driver finds the element that we need
        #myElem = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.ID, 'folder1')))
        
        #element_present = EC.presence_of_element_located((By.ID, 'folder1'))
        #WebDriverWait(driver, 1000).until(element_present)
        
        #print("Page loaded. Parsing with BeautifulSoup.")
        #game_page = BeautifulSoup(driver.page_source) # parse page with beautifulsoup    
    
        # make entry for each game item on page
        
        #game_entries = game_page.find_all('item')
        game_entries = driver.find_elements_by_tag_name('item')
        
        print("Page loaded. Processing.")
        ##### Process Each Game #####
        
        # for each game found on the page:
        for entry in game_entries:
            
            elementHTML = entry.get_attribute('outerHTML') #gives exact HTML content of the element
            elementSoup = BeautifulSoup(elementHTML,'html.parser')
            
            # make an empty dataframe for this item
            this_game_ratings = pd.DataFrame(columns=['BGGId'])
            
            # get the item id and item name
            item_id = int(elementSoup.find('item')['id'])
            item_name = elementSoup.find('name')['value']
        
            # get the list of ratings
            list_of_ratings = elementSoup.find_all('comment')
            
            # set up empty list to store the ratings found on this page
            bggid, names, ratings, comment, usernames = [], [], [], [], []
            
            # for each item in the list of ratings:
            for rating in list_of_ratings:
                # add the raw rating to the raw_ratings dictionary
                # add the item id to the list
                bggid.append(item_id)
                # add the item name to the list
                names.append(item_name)
                # add the rating to the list
                ratings.append(rating['rating'])
                # add the comment to the list
                comment.append(rating['value'])
                # add the username to the list
                usernames.append(rating['username'].strip('_'))
            
            # fill the dataframe for the entry
            this_game_ratings['BGGId'] = bggid    
            this_game_ratings['Name'] = names
            this_game_ratings['Rating'] = ratings
            this_game_ratings['Value'] = comment
            this_game_ratings['Username'] = usernames
            
            # append the entry to the overall user_ratings dataframe
            user_ratings = user_ratings.append(this_game_ratings)
        
        driver.close()
        
        end = time.time()
        print(str(end-start)+" seconds elapsed")
        
    user_ratings.to_pickle('data_dirty/pulled_ratings/user_ratings_comments'+str(item)+'.pkl')
        
    #del game_page
    del driver
    del game_entries
    del elementHTML
    del elementSoup
    del list_of_ratings
    gc.collect() 
        


6
Getting page 1 of 6
Waiting for page to load
Page loaded. Processing.
32.96190071105957 seconds elapsed
Getting page 2 of 6
Waiting for page to load
Page loaded. Processing.
66.31404709815979 seconds elapsed
Getting page 3 of 6
Waiting for page to load
Page loaded. Processing.
102.26646113395691 seconds elapsed
Getting page 4 of 6
Waiting for page to load
Page loaded. Processing.
165.33619785308838 seconds elapsed
Getting page 5 of 6
Waiting for page to load
Page loaded. Processing.
234.79621839523315 seconds elapsed
Getting page 6 of 6
Waiting for page to load
Page loaded. Processing.
298.0183734893799 seconds elapsed
5
Getting page 1 of 5
Waiting for page to load
Page loaded. Processing.
75.5415461063385 seconds elapsed
Getting page 2 of 5
Waiting for page to load
Page loaded. Processing.
154.70605301856995 seconds elapsed
Getting page 3 of 5
Waiting for page to load
Page loaded. Processing.
215.87236714363098 seconds elapsed
Getting page 4 of 5
Waiting for page to load
Page loaded

In [33]:
this_game_ratings

,BGGId,Name,Rating,Value,Username
0,182626,Mistborn: House War,10,,cheshirekat13
1,182626,Mistborn: House War,10,,LaborLawLarry
2,182626,Mistborn: House War,10,,ShakaUVM
3,182626,Mistborn: House War,10,I love this game. Beautiful artwork. Fun to play.,TheBlackThorn
4,182626,Mistborn: House War,10,,pkapera
...,...,...,...,...,...
95,182626,Mistborn: House War,8,,Gryph51
96,182626,Mistborn: House War,8,,Popesixtus
97,182626,Mistborn: House War,8,Control noble houses and win the favor of the ...,TheLordWinter
98,182626,Mistborn: House War,8,,Zaineph


In [34]:
user_ratings

,BGGId,Name,Rating,Value,Username
0,244946,Brook City,10,,Thomas83
1,244946,Brook City,10,"WITH STRETCH GOALS, [45euro], 1 play, premium ...",kukugames
2,244946,Brook City,10,,Vendus
3,244946,Brook City,10,,PanMan
4,244946,Brook City,10,,Buschsoevn
...,...,...,...,...,...
95,182626,Mistborn: House War,8,,Gryph51
96,182626,Mistborn: House War,8,,Popesixtus
97,182626,Mistborn: House War,8,Control noble houses and win the favor of the ...,TheLordWinter
98,182626,Mistborn: House War,8,,Zaineph


# Appendix

### Data Validation

In [ ]:
games=pd.read_pickle('data_dirty/games.pkl')
designers=pd.read_pickle('data_dirty/designers.pkl')
categories=pd.read_pickle('data_dirty/categories.pkl')
mechanics=pd.read_pickle('data_dirty/mechanics.pkl')
artists=pd.read_pickle('data_dirty/artists.pkl')
publishers=pd.read_pickle('data_dirty/publishers.pkl')
subcategories=pd.read_pickle('data_dirty/subcategories.pkl')

In [ ]:
games

In [ ]:
designers.tail()

In [ ]:
categories.tail()

In [ ]:
mechanics.tail()

In [ ]:
artists.tail()

In [ ]:
publishers.tail()

In [ ]:
subcategories.tail()

## Get game ids

In [ ]:
games_ids_current = games['BGGId']
games_ids_current

In [ ]:
games_ids_current.to_pickle('data_dirty/game_ids_current')